In [1]:
import cv2
import matplotlib.pyplot as plt
import time
import cProfile
import numpy as np


# Walker detection with openCV

## Open video and get video info

In [2]:
video_capture = cv2.VideoCapture('resources/TestWalker.mp4')

In [3]:
# From https://www.learnopencv.com/how-to-find-frame-rate-or-frames-per-second-fps-in-opencv-python-cpp/
# Find OpenCV version
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
print major_ver, minor_ver, subminor_ver

# With webcam get(CV_CAP_PROP_FPS) does not work.
# Let's see for ourselves.

if int(major_ver)  < 3 :
    fps = video_capture.get(cv2.cv.CV_CAP_PROP_FPS)
    print "Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps)
else :
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    print "Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps)


# Number of frames to capture
num_frames = 120;


print "Capturing {0} frames".format(num_frames)

# Start time
start = time.time()

# Grab a few frames
for i in xrange(0, num_frames) :
    ret, frame = video_capture.read()


# End time
end = time.time()

# Time elapsed
seconds = end - start
print "Time taken : {0} seconds".format(seconds)

# Calculate frames per second
fps  = num_frames / seconds;
print "Estimated frames per second : {0}".format(fps);


2 4 8
Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): 20.8333333333
Capturing 120 frames
Time taken : 12.6476891041 seconds
Estimated frames per second : 9.48789925278


In [4]:
# cProfile.runctx('video_capture.read()', globals(), locals(), 'profile.prof')
# use snakeviz to read the output of the profiling

## Track walker using difference between frames

Following http://www.codeproject.com/Articles/10248/Motion-Detection-Algorithms

In [4]:
def getSmallGrayFrame(video):
    ret, frame = video.read()
    if not ret:
        return ret, frame
    frameSmall = frame[::4, ::-4]
    gray = cv2.cvtColor(frameSmall, cv2.COLOR_BGR2GRAY)
    return ret, gray


In [5]:
#cv2.startWindowThread()
count = 0
for x in range(200):
    count = count + 1
    print count
    ret1, gray1 = getSmallGrayFrame(video_capture)
    ret2, gray2 = getSmallGrayFrame(video_capture)
    diff = cv2.absdiff(gray1, gray2)
    print np.amax(diff), np.amin(diff)
    print 
    diffThresh = cv2.threshold(diff, 15, 255, cv2.THRESH_BINARY)
    
    kernel = np.ones((3,3),np.uint8)
    erosion = cv2.erode(diffThresh[1],kernel,iterations = 1)
    dilation = cv2.dilate(erosion,kernel,iterations = 1)
    
    color1 = cv2.cvtColor(gray1, cv2.COLOR_GRAY2RGB)
    color1[:,:,0:1] = color1[:,:,0:1]
    
    colorDil = cv2.cvtColor(dilation, cv2.COLOR_GRAY2RGB)
    colorDil[:,:,1:2] = colorDil[:,:,1:2]*0
    
    total = cv2.add(color1, colorDil)
    if not ret1 or not ret2:
        break
    cv2.imshow('Video', total)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Need the cv2.waitKey to update plot
        break

1
193 0

2
201 0

3
214 0

4
212 0

5
233 0

6
220 0

7
219 0

8
197 0

9
193 0

10
195 0

11
197 0

12
159 0

13
197 0

14
131 0

15
157 0

16
96 0

17
81 0

18
77 0

19
59 0

20
27 0

21
30 0

22
28 0

23
27 0

24
26 0

25
27 0

26
23 0

27
33 0

28
24 0

29
30 0

30
31 0

31
30 0

32
32 0

33
28 0

34
27 0

35
34 0

36
32 0

37
30 0

38
26 0

39
23 0

40
65 0

41
85 0

42
91 0

43
110 0

44
119 0

45
136 0

46
134 0

47
160 0

48
163 0

49
183 0

50
198 0

51
198 0

52
183 0

53
185 0

54
188 0

55
223 0

56
197 0

57
218 0

58
242 0

59
217 0

60
224 0

61
203 0

62
193 0

63
208 0

64
196 0

65
203 0

66
198 0

67
188 0

68
194 0

69
197 0

70
197 0

71
163 0

72
185 0

73
168 0

74
165 0

75
161 0

76
160 0

77
156 0

78
108 0

79
112 0

80
132 0

81
105 0

82
113 0

83
116 0

84
126 0

85
143 0

86
156 0

87
141 0

88
151 0

89
123 0

90
133 0

91
157 0

92
156 0

93
85 0

94
117 0

95
142 0

96
144 0

97
105 0

98
94 0

99
77 0

100
93 0

101
64 0

102
72 0

103
84 0

104
75 0


In [5]:
# To close the windows: http://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv#15058451
cv2.waitKey(1000)

cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1